<a href="https://colab.research.google.com/github/ElahehJafarigol/Federated-Learning-GANs/blob/main/SMOTE_based_GANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####**Deep Imbalanced Learning for Weather Data: A Federated Learning Approach**

**The high-level algorithm for the Conditional Generative Adversarial Network (CGAN) is as follows:**

1. Load and preprocess the dataset
2. Define the generator and discriminator models
3. Define the CGAN model by combining the generator and discriminator models
4. Train the discriminator by feeding it real and generated data while adjusting its weights to improve classification accuracy
5. Train the generator by generating synthetic data and feeding it into the discriminator while adjusting its weights to maximize the probability of the generated data being classified as real
6. Repeat steps 4 and 5 for a fixed number of epochs
7. Save the trained generator model
8. Use the generator to generate synthetic data
9. Combine the synthetic data with the real data to create a balanced dataset
10. Train a classification model on the balanced dataset to evaluate its performance.

In [ ]:
!pip install --quiet keras==2.9.0
!pip install --quiet tensorflow==2.9.2

In [ ]:
!pip install --quiet imbalanced-learn

In [ ]:
import os
from  IPython import display
import pathlib
import shutil
import tempfile
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.filterwarnings('ignore')
from pandas.core.common import random_state
import pandas as pd
import numpy as np
import random
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing

random_state = 42
random_seed = 42
tf.random.set_seed(42)

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from keras import datasets, layers, models
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras.optimizers import SGD
from keras import backend as K
from keras import datasets, layers, models, metrics
from keras.layers import GaussianNoise
from keras import regularizers

from keras.layers.serialization import activation
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ReLU
from keras.layers import Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers.activation import ReLU
from pandas.core.indexes.datetimes import Resolution
from keras.layers import Concatenate

import imblearn
from imblearn.over_sampling import SMOTE
from collections import Counter

from sklearn.metrics import roc_auc_score, confusion_matrix

from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ReLU
from keras.layers import Dropout
from keras.layers import Input
from keras.models import Model
from keras.layers.activation import ReLU
from pandas.core.indexes.datetimes import Resolution

####**Multiple Datasets**

###Load and preprocess the data

In [ ]:
# Uploading the data in Google colab
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving WeatherAUS.csv to WeatherAUS.csv
User uploaded file "WeatherAUS.csv" with length 14159645 bytes


1. It imports the weather dataset as a pandas dataframe and preprocesses it. It drops columns that are not useful for the model and replaces the binary labels "RainToday" and "RainTomorrow" with 1 for "Yes" and 0 for "No." It also fills in missing data with the mean of the respective feature.

2. It separates the features and the labels into X and y, respectively, and normalizes the features using MinMaxScaler.

In [ ]:
# Import the data file as a dataframe
import io
weather_df = pd.read_csv(io.BytesIO(uploaded['WeatherAUS.csv']))
# Dataset is now stored in a Pandas Dataframe

weather_df.shape

Data = weather_df
Data.RainToday = [1 if each=="Yes" else 0 for each in Data.RainToday]
Data.RainTomorrow = [1 if each=="Yes" else 0 for each in Data.RainTomorrow]
#Data.head()

Data = Data.drop(['Sunshine','Evaporation','Cloud3pm','Cloud9am','RISK_MM','Location','Date','WindGustDir',
       'WindDir9am', 'WindDir3pm'],axis=1)
Data.shape

# replace rest of the nulls with respective means
fill_feat = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed','WindSpeed9am', 'WindSpeed3pm',
             'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm',
             'RainToday', 'RainTomorrow']
for i in fill_feat:
    Data[i].fillna(np.mean(Data[i]),inplace=True)

Data.shape

Data.dropna(inplace=True)

# Separate the features and labels
X = Data.drop('RainTomorrow', axis=1)
y = Data['RainTomorrow']

print("X:", X.shape)
print("y:", y.shape)

# Normalize the features
scalar = preprocessing.MinMaxScaler(feature_range=(0, 12))
norm_data = scalar.fit_transform(X)
X = pd.DataFrame(norm_data, columns=[X.columns])
X = pd.DataFrame(X.reset_index(drop=True))
X.shape

X: (142193, 13)
y: (142193,)


(142193, 13)

3. It imports the SMOTE module from the imblearn library and oversamples the minority class (i.e., samples where it rained tomorrow) in the dataset to balance the classes.

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE

# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

print("X:", X.shape)
print("y:", y.shape)

X = pd.DataFrame(X.reset_index(drop=True))

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

X: (220632, 13)
y: (220632,)
X shape: (220632, 13)
y shape: (220632,)


4. It imports the SMOTE module from the imblearn library and oversamples the minority class (i.e., samples where it rained tomorrow) in the dataset to balance the classes. The discriminator is trained to maximize the probability of correctly classifying real data and minimizing the probability of incorrectly classifying synthetic data, while the generator is trained to maximize the probability of the discriminator incorrectly classifying the synthetic data as real.

5. It defines the CGANs model, which is the combination of the generator and discriminator. The input to the model is a latent vector, and the output is the probability of the data being real.

6. It trains the CGANs model using the adversarial training algorithm. In each epoch, the discriminator and the generator are trained alternately. The discriminator is trained on a combination of real and synthetic data with corresponding labels of 1 and 0, respectively, while the generator is trained to produce synthetic data that the discriminator will classify as real.

7. After training, the generator is saved, and synthetic data is generated by passing a latent vector to the generator's predict() method.

In [ ]:
epochs = 10
batch_size = 64
latent_dim = 13

# Define the generator model
def define_generator(latent_dim):
    model = Sequential()
    model.add(Dense(1024, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))

    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))

    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(256))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))

    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(X.shape[1], activation='sigmoid'))

    # model.summary()
    return model

# Define the discriminator model
def define_discriminator(input_shape):
    model = Sequential()
    model.add(Dense(512, input_dim=input_shape))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.SGD(0.0003, 0.5),
                  metrics=['accuracy'])
    # model.summary()
    return model

# Train the CGANs model
def train(X, y, epochs, batch_size, latent_dim):

    # Remove rows with missing data
    X = X.dropna()

    # Get the set of valid indices
    valid_indices = set(X.index)

    # Define the CGANs model

    # Instantiate the generator
    generator = define_generator(latent_dim)

    discriminator = define_discriminator(X.shape[1])
    cgan_input = Input(shape=(latent_dim,))
    discriminator_output = discriminator(generator(cgan_input))
    cgan = Model(cgan_input, discriminator_output)
    cgan.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(0.0003, 0.5))

    # Train the CGANs model
    for epoch in range(epochs):

    # Train the discriminator
        try:
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            generated_data = generator.predict(noise)

            # Sample indices from the valid range of indices in the dataset
            real_indices = np.random.choice(list(valid_indices), size=batch_size, replace=False)
            real_data = X.loc[real_indices]
            combined_data = np.concatenate([generated_data, real_data])
            labels = np.concatenate([np.zeros((batch_size, 1)), np.ones((batch_size, 1))])
            d_loss = discriminator.train_on_batch(combined_data, labels)
        except KeyError as e:
            print(f"Skipping row {id} due to KeyError: {e}")
            continue

        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        misleading_targets = np.ones((batch_size, 1))
        g_loss = cgan.train_on_batch(noise, misleading_targets)

        # Print the progress
        print(f"Epoch {epoch} Discriminator Loss: {d_loss} Generator Loss: {g_loss}")

In [ ]:
train(X, y, epochs = 20, batch_size = 128, latent_dim = 13)

4/4 [==============================] - 3s 2ms/step
Epoch 0 Discriminator Loss: [0.587205171585083, 0.5859375] Generator Loss: 0.6613437533378601
4/4 [==============================] - 0s 2ms/step
Epoch 1 Discriminator Loss: [0.578474760055542, 0.53125] Generator Loss: 0.6616266369819641
4/4 [==============================] - 0s 2ms/step
Epoch 2 Discriminator Loss: [0.5573899745941162, 0.57421875] Generator Loss: 0.656248927116394
4/4 [==============================] - 0s 2ms/step
Epoch 3 Discriminator Loss: [0.5455784201622009, 0.56640625] Generator Loss: 0.6468992829322815
4/4 [==============================] - 0s 2ms/step
Epoch 4 Discriminator Loss: [0.5292125940322876, 0.515625] Generator Loss: 0.6535479426383972
4/4 [==============================] - 0s 2ms/step
Epoch 5 Discriminator Loss: [0.5347127318382263, 0.578125] Generator Loss: 0.6493207216262817
4/4 [==============================] - 0s 2ms/step
Epoch 6 Discriminator Loss: [0.5448351502418518, 0.5078125] Generator Loss: 0.

In [ ]:
original_indices = set(Data.index)
current_indices = set(X.index)

missing_indices = original_indices - current_indices
print(missing_indices)

set()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

# Calculate the class ratio in the balanced dataset
class_ratio = sum(y_train == 0) / len(y_train)
class_ratio

0.500393756550806

In [ ]:
# Save the generator
generator = define_generator(latent_dim)

# Use the generator to create synthetic data
synthetic_data = generator.predict(np.random.normal(0, 1, (len(X_train), latent_dim)))

# Combine the real and synthetic data to create a balanced dataset
X_balanced = np.concatenate([X_train, synthetic_data])
y_balanced = np.concatenate([y_train, np.zeros(len(X_train))])

# Calculate the class ratio in the balanced dataset
class_ratio = sum(y_balanced == 0) / len(y_balanced)
print(class_ratio)

print(X_balanced.shape)
print(X_train.shape)

5516/5516 [==============================] - 7s 1ms/step
0.7501968782754029
(353010, 13)
(176505, 13)


#####Training Federated Learning

In [ ]:
# define data shape and number of classes
input_shape = (13,)
num_classes = 2
num_test_samples = len(X_test)
num_train_samples = len(X_train)
Batch_size = 64
communication_round = 10
Learning_rate = 0.001
Momentum = 0.9
Epochs = 30
data_shape = (13,)

In [ ]:
def create_clients(X_train, y_train, num_clients, initial='client'):
    # create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    # randomize the data before splitting the data between clients
    indices = np.random.permutation(len(X_train))
    X_shuffled = X_train[indices]
    y_shuffled = y_train[indices]

    # shard data and place at each client
    size = len(X_train) // num_clients
    remainder = len(X_train) % num_clients
    shards = [(X_shuffled[i:i+size], y_shuffled[i:i+size]) for i in range(0, size*num_clients, size)]
    if remainder:
        shards[-1] = (np.concatenate([shards[-1][0], X_shuffled[-remainder:]]),
                      np.concatenate([shards[-1][1], y_shuffled[-remainder:]]))

    # number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    # create dictionary of clients and their shards
    clients = {}
    for i in range(len(client_names)):
        # create dictionary entry with client name and data shape
        clients[client_names[i]] = (shards[i][0].shape, shards[i][1].shape)

    return clients

def batch_data(data_shard, batchsize=Batch_size):
    # unpack data shard into data and labels arrays
    X, y = data_shard

    # create a tensorflow dataset object from the data and labels
    dataset = tf.data.Dataset.from_tensor_slices((X, y))

    # shuffle and batch the dataset
    return dataset.shuffle(len(y)).batch(batchsize)

# create clients from training data and associated labels
X_train = X_balanced
y_train = y_balanced
clients = create_clients(X_train, y_train, num_clients=10, initial='client')
for client in clients:
    print(f"{client}: X={clients[client][0]}, y={clients[client][1]}")

# You can access the shards for each client by indexing the dictionary returned
# by create_clients(), like so:
client_1_shard = clients['client_1']
client_1_shard

# create a batched dataset for each client's data shard
clients_batched = {}
for client in clients:
    client_data_shard = (X_train, y_train)
    batched_data = batch_data(client_data_shard)
    clients_batched[client] = batched_data

#process and batch the test set
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

# Federated Averaging: aggregation method
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the batch_size
    batch_size = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*batch_size
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*batch_size
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

def sum_scaled_weights(scaled_weight_list):
    # Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad

def test_model(X_test, Y_test, model, communication_round):
    cce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    logits = model.predict(X_test, batch_size=Batch_size)
    logits = tf.squeeze(logits, axis=1) # remove last dimension from logits
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.round(logits), Y_test)
    auc = roc_auc_score(Y_test, logits)
    tn, fp, fn, tp = confusion_matrix(Y_test, tf.round(logits)).ravel()
    g_mean = np.sqrt(tp/(tp+fn)*tn/(tn+fp))
    print('communication_round: {} | global_accuracy: {:.3%} | global_loss: {} | global_AUC: {:.3%} | global_G-mean: {:.3%}'.format(communication_round, acc, loss, auc, g_mean))
    return acc, loss, auc, g_mean

client_1: X=(35301, 13), y=(35301,)
client_2: X=(35301, 13), y=(35301,)
client_3: X=(35301, 13), y=(35301,)
client_4: X=(35301, 13), y=(35301,)
client_5: X=(35301, 13), y=(35301,)
client_6: X=(35301, 13), y=(35301,)
client_7: X=(35301, 13), y=(35301,)
client_8: X=(35301, 13), y=(35301,)
client_9: X=(35301, 13), y=(35301,)
client_10: X=(35301, 13), y=(35301,)


In [ ]:
epochs = Epochs
learning_rate = Learning_rate
momentum = Momentum
batch_size = Batch_size

# Define the optimizer and loss function
optimizer = keras.optimizers.SGD(learning_rate=0.01,
                                 momentum = momentum,
                                 nesterov = False)
loss = "binary_crossentropy"

metrics = [keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      ]

class MyModel:
    @staticmethod
    def build(shape, classes):
        model = tf.keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(13,)),
        layers.GaussianNoise(stddev = 0.5),
        layers.Dense(128, activation='relu'),
        layers.GaussianNoise(stddev = 0.3),
        layers.Dense(64, activation='relu'),
        layers.GaussianNoise(stddev = 0.1),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')
        ])
        return model

In [ ]:
smlp_global = MyModel()
global_model = smlp_global.build(shape=(13,), classes=2)
global_model.summary()

global_acc_list = []
global_loss_list = []
global_auc_list = []
global_gmean_list = []

# commence global training loop
for communication_round in range(communication_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    # initial list to collect local model weights after scaling
    scaled_local_weight_list = list()

    # randomize client data - using keys
    client_names = list(clients_batched.keys())
    random.shuffle(client_names)

    # loop through each client and create new local model
    for client in client_names:
        smlp_local = MyModel()
        local_model = smlp_local.build(shape=(13,), classes=2)
        #local_model = smlp_local.build()
        local_model.compile(loss=loss,
                            optimizer=optimizer,
                            metrics=metrics)

        # set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        # fit local model with client's data
        model_history = local_model.fit(clients_batched[client], epochs=epochs, verbose=1)

        # evaluate local model on test data
        # test_loss, test_accuracy = local_model.evaluate(X_test, y_test)
        # print(f'Local model {client} - test_loss: {test_loss} - test_accuracy: {test_accuracy}')

        # scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        # clear session to free memory after each communication round
        K.clear_session()

    # to get the average over all the local model, we simply take the sum of the scaled weights
    avg_grad = sum_scaled_weights(scaled_local_weight_list)

    # update global model
    global_model.set_weights(avg_grad)

    # test global model and print out metrics after each communications round
    for X_test, y_test in test_batched:
        global_acc, global_loss, global_auc, global_gmean = test_model(X_test, y_test, global_model, communication_round)
        global_acc_list.append(global_acc)
        global_loss_list.append(global_loss)
        global_auc_list.append(global_auc)
        global_gmean_list.append(global_gmean)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 256)               3584      
                                                                 
 gaussian_noise (GaussianNoi  (None, 256)              0         
 se)                                                             
                                                                 
 dense_16 (Dense)            (None, 128)               32896     
                                                                 
 gaussian_noise_1 (GaussianN  (None, 128)              0         
 oise)                                                           
                                                                 
 dense_17 (Dense)            (None, 64)                8256      
                                                                 
 gaussian_noise_2 (GaussianN  (None, 64)              

####**Single Dataset**

###Load and preprocess the data

In [ ]:
# Uploading the data in Google colab
from google.colab import files

uploaded_1 = files.upload()

for fn in uploaded_1.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded_1[fn])))

In [ ]:
# Import the data file as a dataframe
import io
weather_df = pd.read_csv(io.BytesIO(uploaded_1['Client1.csv']))
# Dataset is now stored in a Pandas Dataframe

weather_df.shape

In [ ]:
Data = weather_df
Data.RainToday = [1 if each=="Yes" else 0 for each in Data.RainToday]
Data.RainTomorrow = [1 if each=="Yes" else 0 for each in Data.RainTomorrow]
#Data.head()

Data = Data.drop(['Sunshine','Evaporation','Cloud3pm','Cloud9am','RISK_MM','Location','Date','WindGustDir',
       'WindDir9am', 'WindDir3pm'],axis=1)
Data.shape

# replace rest of the nulls with respective means
fill_feat = ['MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed','WindSpeed9am', 'WindSpeed3pm',
             'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm',
             'RainToday', 'RainTomorrow']
for i in fill_feat:
    Data[i].fillna(np.mean(Data[i]),inplace=True)

Data.shape
Data.dropna(inplace=True)

# Separate the features and labels
X = Data.drop('RainTomorrow', axis=1)
y = Data['RainTomorrow']

print("X:", X.shape)
print("y:", y.shape)

# Normalize the features
scalar = preprocessing.MinMaxScaler(feature_range=(0, 12))
norm_data = scalar.fit_transform(X)
X = pd.DataFrame(norm_data, columns=[X.columns])
X = pd.DataFrame(X.reset_index(drop=True))

original_indices = set(Data.index)
current_indices = set(X.index)

missing_indices = original_indices - current_indices
print(missing_indices)

X_single = X
y_single = y

counter = Counter(y_single)
print(counter)

#####Data Augmentation

In [ ]:
import imblearn
from imblearn.over_sampling import SMOTE

# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

print("X:", X.shape)
print("y:", y.shape)

X = pd.DataFrame(X.reset_index(drop=True))

In [ ]:
train(X, y, epochs = 20, batch_size = 128, latent_dim = 13)

# Save the generator
generator_0 = define_generator(latent_dim)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

# Calculate the class ratio in the balanced dataset
class_ratio = sum(y_train == 0) / len(y_train)
class_ratio

In [ ]:
# Use the generator to create synthetic data
synthetic_data = generator_0.predict(np.random.normal(0, 1, (len(X_train), latent_dim)))

# Combine the real and synthetic data to create a balanced dataset
X_balanced = np.concatenate([X_train, synthetic_data])
y_balanced = np.concatenate([y_train, np.zeros(len(X_train))])

print("X_train", X_train.shape)
print("X_balanced", X_balanced.shape)

print("X_balanced:", X_balanced.shape)
print("y_balanced:", y_balanced.shape)

#####Centralized Training

In [ ]:
# define data shape and number of classes
input_shape = (13,)
num_classes = 2
num_test_samples = len(X_test)
num_train_samples = len(X_train)
Batch_size = 64
communication_round = 10
Learning_rate = 0.001
Momentum = 0.9
Epochs = 30
data_shape = (13,)

In [ ]:
epochs = Epochs
learning_rate = Learning_rate
momentum = Momentum
batch_size = Batch_size

# Define the optimizer and loss function
optimizer = keras.optimizers.SGD(learning_rate=0.01,
                                 momentum = momentum,
                                 nesterov = False)
loss = "binary_crossentropy"

metrics = [keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      ]

model = tf.keras.Sequential([
layers.Dense(128, activation='relu', input_shape=(13,)),
layers.Dense(64, activation='relu'),
layers.Dense(32, activation='relu'),
layers.Dense(1, activation='sigmoid')
])


model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)

model.fit(X_balanced, y_balanced, epochs=Epochs, batch_size=Batch_size)

In [ ]:
Evaluation = model.evaluate(X_test, y_test, verbose = 1)